# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Описание проекта
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.
Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

## Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

    сalls — количество звонков,
    minutes — суммарная длительность звонков в минутах,
    messages — количество sms-сообщений,
    mb_used — израсходованный интернет-трафик в Мб,
    is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## Разбейте данные на выборки

In [5]:
# выведем столбец is_ultra как целевой
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

In [6]:
#разделим выборку на тренироваочную и валидационную
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.40, random_state=12345, stratify= target)

In [7]:
features_train.shape

(1928, 4)

In [8]:
features_valid.shape

(1286, 4)

In [9]:
# разделим валидационную выборку еще пополом чтобы пролучить теустовую выборку
features_valid, features_test, target_valid,target_test = train_test_split(
    features_valid, target_valid, test_size=0.50, random_state=12345, stratify = target_valid)

In [10]:
features_valid.shape

(643, 4)

In [11]:
features_test.shape

(643, 4)

Данные были разбиты по принципу 60/20/20.

    features_train, target_train - обучающая выборка 60% от набора данных
    features_valid, target_valid - валидационная выборка 20% от набора 
    feаatures_test, target_test - тестовая выборка 20% от набора 


## Исследуйте модели

In [12]:
#посчитаем точность и найдем лучшую глубину деревом решений
best_accuracy = 0
best_depth = 0
for depth in range(1,10):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    accuracy = model.score(features_valid, target_valid)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth = depth
print("Глубина дерева:", best_depth, "    Точность:", best_accuracy)

Глубина дерева: 8     Точность: 0.80248833592535


При использовании модели дерева решений лучшая глубина дерева для данной вывыборки  8. Точность при этом 0.80248833592535

In [13]:
#посчитаем точность логистической регрессии
model = LogisticRegression()
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)
print("Точность:", accuracy)

Точность: 0.7387247278382582


точность логистической регрессии получилась 0.7387247278382582

In [14]:
#посчитаем качество и найдем лучшие параметры  в случайном лее
best_accuracy = 0
best_depth = 0
for depth in range(1,10):
    for est in range(5,50,5):
        model = RandomForestClassifier(max_depth=depth, n_estimators=est, random_state=12345)
        model.fit(features_train, target_train)
        accuracy = model.score(features_valid, target_valid)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_depth = depth
            best_est = est
print("Глубина дерева:", best_depth, "    Количество деревьев:", best_est,
      "  Точность:", best_accuracy)


Глубина дерева: 9     Количество деревьев: 40   Точность: 0.8211508553654744


для модели случаный лес лучшими параметрами для данной выборки оказались : Глубина дерева - 9, Количество деревьев- 40

Точность при этом на тенировочной выборке полусилась 0.8211508553654744

## Проверьте модель на тестовой выборке

In [15]:
# проверка логистической регресси на тестовой выборке
model = LogisticRegression()
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Точность:", accuracy)

Точность: 0.7465007776049767


In [16]:
# проверка дерева решений на тестовой выборке
model = DecisionTreeClassifier(max_depth=8, random_state=12345)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Точность:", accuracy)

Точность: 0.7838258164852255


In [17]:
# проверка случайный лес  на тестовой выборке
model = RandomForestClassifier(max_depth=9, n_estimators=40, random_state=12345)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Точность:", accuracy)

Точность: 0.8087091757387247


Лучшие показатели показала модель случайный лес с точностью  0.8087091757387247

## (бонус) Проверьте модели на адекватность

In [18]:
dmb = DummyClassifier(strategy="stratified" ,random_state=12345)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Точность наивного прогноза :", acc)
print("Точность нашего прогноза :", accuracy)

Точность наивного прогноза : 0.5567651632970451
Точность нашего прогноза : 0.8087091757387247


In [19]:
dmb = DummyClassifier(strategy='most_frequent', random_state=12345)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Точность наивного прогноза :", acc)
print("Точность нашего прогноза :", accuracy)

Точность наивного прогноза : 0.6936236391912908
Точность нашего прогноза : 0.8087091757387247


In [20]:
dmb = DummyClassifier(strategy='uniform', random_state=12345)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Точность наивного прогноза :", acc)
print("Точность нашего прогноза :", accuracy)

Точность наивного прогноза : 0.49455676516329705
Точность нашего прогноза : 0.8087091757387247
